### Find Ngram Youtube Link

This notebook was used to find youtube videos on the ngrams selected for the lesson.

In [83]:
import os
import pickle
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [84]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [85]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

# main or additional result
main_search = False  # True, False for additional result. Default True

In [86]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [87]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [88]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [89]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [90]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/3-Find Ngram Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [91]:
## Master File
#path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Data/Deployment/Talk Time 5 Master File.xlsx"
#sheet = "Sheet6"  # Sheet1
#file_ext = "Sixgram"

In [92]:
# search file path folder
ngram = "Twogram"
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/\
Manuel Select/Revised Ngrams All in One.xlsx"
sheet = "Sheet2"
file_ext = ngram
# youtube tool output path
out_path = "/home/kurubal/Downloads"

#### Read Data File

In [93]:
if main_search:
    df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
else:
    df_file = pd.read_excel(f"{out_path}/{ngram}_Re-Search_Result.xlsx")
    
df_file

,not_result
0,fikir değil
1,kadar yeter
2,nerede olduğunu
3,yardım et
4,yaptığını sanıyorsun
...,...
77,neydi böyle
78,eder misin
79,seni seviyorum
80,yok değil


In [94]:
if main_search:
    search_list = list(set(df_file.loc[:,f"{ngram.lower()}"].dropna()))
else:
    search_list = list(set(df_file["not_result"]))
    
len(search_list)

82

In [95]:
if main_search:
    disable_video_id_list = []
else:
    with open("disable_videoid.pkl", "rb") as file:
        disable_video_id_list = pickle.load(file)

In [96]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [97]:
#sent_count = len(df_youtube_sentence)
#sent_count

In [98]:
#df_youtube_sentence = df_youtube_sentence.sample(sent_count, random_state=52)
#df_youtube_sentence.reset_index(drop=True, inplace=True)
#df_youtube_sentence

In [99]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
5166,00:00:00.079,00:00:03.682,bu dizinin betimlemesi trt tarafından sesli be...,VpRGQkSZgrU
5167,00:00:03.737,00:00:05.706,erişim wwwseslibetimlemedernegicom,VpRGQkSZgrU
5168,00:00:07.348,00:00:09.332,cahit deyince,VpRGQkSZgrU
5169,00:00:09.601,00:00:12.133,cahit deyince aklıma sular yıldızlar,VpRGQkSZgrU
5170,00:00:13.030,00:00:14.711,alınlarında şafak örülü,VpRGQkSZgrU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [100]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

<ipython-input-100-a8911ca4399e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second
<ipython-input-100-a8911ca4399e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])


In [101]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

<ipython-input-101-aa25c83efaa1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
<ipython-input-101-aa25c83efaa1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())


,start_time,end_time,sentence,video_id
5166,0.079,3.682,bu dizinin betimlemesi trt tarafından sesli be...,VpRGQkSZgrU
5167,3.737,5.706,erişim wwwseslibetimlemedernegicom,VpRGQkSZgrU
5168,7.348,9.332,cahit deyince,VpRGQkSZgrU
5169,9.601,12.133,cahit deyince aklıma sular yıldızlar,VpRGQkSZgrU
5170,13.030,14.711,alınlarında şafak örülü,VpRGQkSZgrU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


#### Find Ngram In Youtube

In [102]:
if main_search:
    df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
else:
    df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", 30)
    
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,fikir değil,4788.254,4795.658,yani ben sabaha kadar düşündüm de aslında bu b...,P0U8Cuf5lGM
1,fikir değil,5.360,9.880,hızırın beni sattığınızı düşünmesi kötü bir fi...,Eq6JAzDH7_s
2,fikir değil,296.580,298.880,aslında fena fikir değil ağzını burnunu kırmak da,kosDbw1NUPM
3,fikir değil,1594.907,1596.617,aslında kötü de fikir değil,qWG21pLc2fw
4,fikir değil,1260.640,1263.680,hı aslında hiç fena fikir değil yani,TOmTZnJE7SM
...,...,...,...,...,...
2286,yardım etmek,2118.473,2121.445,ben yalnız yardım etmek istedim,IHwXUP6dMIQ
2287,yardım etmek,192.770,197.990,yardım etmek isteyen böyle aşırı merhametli in...,W5VdzH65XRU
2288,yardım etmek,2962.255,2965.224,eski bir çalışanımıza yardım etmek isteriz tabii,x8yoLCYSzVw
2289,yardım etmek,3825.403,3828.868,şerefsiz polise yardım etmek vatandaşlık görev...,0t1D3P6NYYM


In [103]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,fikir değil,4794.473360,4795.658000,yani ben sabaha kadar düşündüm de aslında bu b...,P0U8Cuf5lGM
1,fikir değil,8.893818,9.880000,hızırın beni sattığınızı düşünmesi kötü bir fi...,Eq6JAzDH7_s
2,fikir değil,297.143265,297.753469,aslında fena fikir değil ağzını burnunu kırmak da,kosDbw1NUPM
3,fikir değil,1595.857000,1596.617000,aslında kötü de fikir değil,qWG21pLc2fw
4,fikir değil,1262.244444,1263.342222,hı aslında hiç fena fikir değil yani,TOmTZnJE7SM
...,...,...,...,...,...
2286,yardım etmek,2119.431710,2120.773903,ben yalnız yardım etmek istedim,IHwXUP6dMIQ
2287,yardım etmek,192.770000,193.550000,yardım etmek isteyen böyle aşırı merhametli in...,W5VdzH65XRU
2288,yardım etmek,2963.553938,2964.419896,eski bir çalışanımıza yardım etmek isteriz tabii,x8yoLCYSzVw
2289,yardım etmek,3826.215109,3826.973078,şerefsiz polise yardım etmek vatandaşlık görev...,0t1D3P6NYYM


In [104]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,fikir değil,4793.873360,4796.258000,yani ben sabaha kadar düşündüm de aslında bu b...,P0U8Cuf5lGM
1,fikir değil,8.293818,10.480000,hızırın beni sattığınızı düşünmesi kötü bir fi...,Eq6JAzDH7_s
2,fikir değil,296.543265,298.353469,aslında fena fikir değil ağzını burnunu kırmak da,kosDbw1NUPM
3,fikir değil,1595.257000,1597.217000,aslında kötü de fikir değil,qWG21pLc2fw
4,fikir değil,1261.644444,1263.942222,hı aslında hiç fena fikir değil yani,TOmTZnJE7SM
...,...,...,...,...,...
2286,yardım etmek,2118.831710,2121.373903,ben yalnız yardım etmek istedim,IHwXUP6dMIQ
2287,yardım etmek,192.170000,194.150000,yardım etmek isteyen böyle aşırı merhametli in...,W5VdzH65XRU
2288,yardım etmek,2962.953938,2965.019896,eski bir çalışanımıza yardım etmek isteriz tabii,x8yoLCYSzVw
2289,yardım etmek,3825.615109,3827.573078,şerefsiz polise yardım etmek vatandaşlık görev...,0t1D3P6NYYM


In [105]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,fikir değil,4794,4796,yani ben sabaha kadar düşündüm de aslında bu b...,P0U8Cuf5lGM
1,fikir değil,8,10,hızırın beni sattığınızı düşünmesi kötü bir fi...,Eq6JAzDH7_s
2,fikir değil,297,298,aslında fena fikir değil ağzını burnunu kırmak da,kosDbw1NUPM
3,fikir değil,1595,1597,aslında kötü de fikir değil,qWG21pLc2fw
4,fikir değil,1262,1264,hı aslında hiç fena fikir değil yani,TOmTZnJE7SM
...,...,...,...,...,...
2286,yardım etmek,2119,2121,ben yalnız yardım etmek istedim,IHwXUP6dMIQ
2287,yardım etmek,192,194,yardım etmek isteyen böyle aşırı merhametli in...,W5VdzH65XRU
2288,yardım etmek,2963,2965,eski bir çalışanımıza yardım etmek isteriz tabii,x8yoLCYSzVw
2289,yardım etmek,3826,3828,şerefsiz polise yardım etmek vatandaşlık görev...,0t1D3P6NYYM


In [106]:
#df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
#df_word_group_time_loc

In [107]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,ne,16
1,bir,5
2,sen,5
3,değil,5
4,var,4
...,...,...
84,tamam,1
85,ya,1
86,seviyorum,1
87,bahsediyorsun,1


In [108]:
if main_search:
    df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link.xlsx", index=False)  ########*****
else:
    df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link2.xlsx", index=False)

#### Copy Move And Delete

In [109]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_With_{word_end}_Word_Youtube_Link*.xlsx")   ########*****
output_file

['Turkish_Fourgram_With_1000_Word_Youtube_Link2.xlsx',
 'Turkish_Threegram_With_1000_Word_Youtube_Link2.xlsx',
 'Turkish_Twogram_With_1000_Word_Youtube_Link2.xlsx']

In [110]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [111]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Tool

In [112]:
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/\
Manuel Select/Revised Ngrams All in One.xlsx"
sheet = "Sheet2"

In [113]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,NaN,NaN
2,bana yardım eder misin,yardım eder misin,eder misin
3,bana yardım eder misin,yardım eder misin,yardım eder
4,başka bir şey daha,başka bir şey,bir şey
...,...,...,...
495,NaN,çok kötü bir,çok kötü
496,NaN,bir şey olmaz,NaN
497,NaN,şuna bir bak,bir bak
498,NaN,şuna bir bak,NaN


In [114]:
if main_search:
    df_twogram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link.xlsx")
else:
    df_twogram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link2.xlsx")
    
df_twogram_link

,search_string,start_time,end_time,sentence,video_id
0,fikir değil,4794,4796,yani ben sabaha kadar düşündüm de aslında bu b...,P0U8Cuf5lGM
1,fikir değil,8,10,hızırın beni sattığınızı düşünmesi kötü bir fi...,Eq6JAzDH7_s
2,fikir değil,297,298,aslında fena fikir değil ağzını burnunu kırmak da,kosDbw1NUPM
3,fikir değil,1595,1597,aslında kötü de fikir değil,qWG21pLc2fw
4,fikir değil,1262,1264,hı aslında hiç fena fikir değil yani,TOmTZnJE7SM
...,...,...,...,...,...
2286,yardım etmek,2119,2121,ben yalnız yardım etmek istedim,IHwXUP6dMIQ
2287,yardım etmek,192,194,yardım etmek isteyen böyle aşırı merhametli in...,W5VdzH65XRU
2288,yardım etmek,2963,2965,eski bir çalışanımıza yardım etmek isteriz tabii,x8yoLCYSzVw
2289,yardım etmek,3826,3828,şerefsiz polise yardım etmek vatandaşlık görev...,0t1D3P6NYYM


In [115]:
set(df_file["twogram"].dropna()).difference(set(df_twogram_link["search_string"]))

{'ama yine',
 'başka bir',
 'benim için',
 'bir bak',
 'bir dakika',
 'bir fikir',
 'bir kez',
 'bir sorun',
 'bir tane',
 'bir yer',
 'bir şeye',
 'bir şeyler',
 'bu da',
 'bu iyi',
 'bu kadar',
 'bu senin',
 'bu çok',
 'bunu yapmak',
 'böyle bir',
 'büyük bir',
 'da ne',
 'da neydi',
 'daha iyi',
 'daha var',
 'daha önce',
 'demek oluyor',
 'doğru mu',
 'en son',
 'etmek istiyorum',
 'evet ben',
 'evet mi',
 'fark eder',
 'fikrim var',
 'güzel bir',
 'hem de',
 'her şey',
 'herhangi bir',
 'hiçbir şey',
 'ihtiyacın olursa',
 'iyi bir',
 'iyi değil',
 'iyi olur',
 'için teşekkürler',
 'kadar da',
 'kadar güzel',
 'kadar uzun',
 'kontrol altında',
 'kötü değil',
 'ne kadar',
 'ne olmaz',
 'ne yaptığını',
 'ne yazık',
 'ne yok',
 'neden bu',
 'neydi böyle',
 'o da',
 'o zaman',
 'olabilir miyim',
 'olursa olsun',
 'sizin için',
 'sorun var',
 'söylemek istiyorum',
 'söyleyeyim mi',
 'tam olarak',
 'teşekkür ederim',
 'uzun zaman',
 'var biliyor',
 'var burada',
 'var mı',
 've ben',
 'v

In [116]:
if main_search:
    df_threegram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link.xlsx")
else:
    df_threegram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link2.xlsx")
    
df_threegram_link

,search_string,start_time,end_time,sentence,video_id
0,peki ya sen,1085,1088,peki ya sen,i9AZdyMTPXs
1,peki ya sen,1274,1277,peki ya sen,i9AZdyMTPXs
2,peki ya sen,563,566,peki ya sen,CSr36NrNoWM
3,peki ya sen,387,390,peki ya sen,Qs1I7HxXYlg
4,peki ya sen,2402,2405,peki ya sen şu teklifi şöyle mi yapsak acaba,tU1Mv8UkyR4
...,...,...,...,...,...
2029,böyle bir şey,197,199,ya böyle bir şey yok,5EsyiXMKOPs
2030,böyle bir şey,495,496,allaha şükür bugüne kadar böyle bir şey başımı...,6rqwPW97HiY
2031,böyle bir şey,895,897,eskiden gitmiyordum ben mesela böyle bir şey v...,h93wXNLXmYU
2032,böyle bir şey,66,68,alıyorum şu hayır öyle değilim çok amatörüm il...,duaQwKTmgvE


In [117]:
set(df_file["threegram"].dropna()).difference(set(df_threegram_link["search_string"]))

{'ama yine de',
 'bekle bir dakika',
 'ben de bir',
 'benim için bir',
 'benim için çok',
 'bir fikrim yok',
 'bir kez daha',
 'bir tane daha',
 'bir yer var',
 'bir şey buldun',
 'bir şey daha',
 'bir şey değil',
 'bir şey olmaz',
 'bir şey söylemek',
 'bir şey söyleyeyim',
 'bir şey yoktur',
 'bir şeyler var',
 'bu da ne',
 'bu da neydi',
 'bu doğru mu',
 'bu kadar uzun',
 'bu kadar çok',
 'bu ne demek',
 'bu çok güzel',
 'da ne böyle',
 'da neydi böyle',
 'daha iyi bir',
 'daha iyi olur',
 'daha önce de',
 'diye bir şey',
 'dur bir dakika',
 'en son ne',
 'gibi bir şey',
 'görmek çok güzel',
 'güzel değil mi',
 'hem de çok',
 'her şey iyi',
 'herhangi bir şey',
 'hiç bir fikrim',
 'hiç böyle bir',
 'hiçbir fikrim yok',
 'hiçbir şey yok',
 'istediğim bir şey',
 'istediğin bir şey',
 'iyi bir şey',
 'için bir şey',
 'için teşekkür ederim',
 'için çok teşekkür',
 'için çok önemli',
 'için özür dilerim',
 'kendine iyi bak',
 'kötü bir şey',
 'ne kadar güzel',
 'ne tür bir',
 'ne yazık k

In [118]:
if main_search:
    df_fourgram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link.xlsx")
else:
    df_fourgram_link = pd.read_excel(f"{path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link2.xlsx")
    
df_fourgram_link

,search_string,start_time,end_time,sentence,video_id
0,ne demek biliyor musun,5180,5182,düğün ne demek biliyor musun kızımın evleniyor,RPJg_lUfHvI
1,ne demek biliyor musun,1944,1947,bir insanın kalbinde ikinci sırada olmak ne de...,6eBEz91Zwh8
2,ne demek biliyor musun,299,302,bu akşam şirin ablasında kalıyor bu ne demek b...,hMKOOivdGiA
3,ne demek biliyor musun,30,32,sen bir kadının kocasından korkması ne demek b...,mDMEFdbxRRY
4,ne demek biliyor musun,4664,4667,eline bile dokunamaması ne demek biliyor musun,oYhTDRJVx7k
...,...,...,...,...,...
774,başka bir şey var,6235,6238,başka bir şey var lakin saklarsın,pCyZEfurYNQ
775,başka bir şey var,4486,4488,kesin bunların kafasında başka bir şey var,UDlsIakb9Kg
776,başka bir şey var,535,537,başka bir şey var mı hadi bey bunların dışında...,wsUGDArhZFg
777,başka bir şey var,813,816,söylemek istediğin başka bir şey var mı,vgHcue8Uu9I


In [119]:
set(df_file["fourgram"].dropna()).difference(set(df_fourgram_link["search_string"]))

{'başka bir şey değil',
 'benim için çok önemli',
 'bir tane daha var',
 'bir yer var mı',
 'bir şey buldun mu',
 'bir şey değil mi',
 'bir şey söylemek istiyorum',
 'bir şey var mı',
 'bir şey yok mu',
 'bir şeyler var mı',
 'bu da ne böyle',
 'bu da ne demek',
 'bu da neydi böyle',
 'bu hiç iyi değil',
 'bu iyi bir fikir',
 'bunu sen mi yaptın',
 'bunu yapmak zorunda değilsin',
 'da ne demek oluyor',
 'daha önce hiç böyle',
 'daha önce hiç görmedim',
 'diye bir şey yok',
 'en son ne zaman',
 'evet mi hayır mı',
 'hiç bir fikrim yok',
 'istediğim bir şey var',
 'istediğin bir şey var',
 'iyi bir fikir değil',
 'için çok teşekkür ederim',
 'kadar da kötü değil',
 'ne kadar güzel bir',
 'ne olur ne olmaz',
 'ne var ne yok',
 'neden bu kadar uzun',
 'neden bu kadar çok',
 'onun gibi bir şey',
 'pek bir şey yok',
 'sadece sen ve ben',
 'sana yardım etmek istiyorum',
 'seni görmek ne güzel',
 'seni görmek çok güzel',
 'sizin için ne yapabilirim',
 'yanlış bir şey mi',
 'çok güzel değil mi'

In [120]:
disable_videoid = list(set(df_twogram_link["video_id"])) + list(set(df_threegram_link["video_id"])) + list(set(df_fourgram_link["video_id"]))
#disable_videoid

In [121]:
if main_search:
    with open("disable_videoid.pkl", "wb") as file:
        pickle.dump(disable_videoid, file)
else:
    pass 

In [122]:
df_twogram_link_reassign = df_twogram_link.copy()
df_threegram_link_reassign = df_threegram_link.copy()
df_fourgram_link_reassign = df_fourgram_link.copy()

In [123]:
df_twogram_link_reassign["start_time"] = 1
df_twogram_link_reassign["end_time"] = 2
df_threegram_link_reassign["start_time"] = 1
df_threegram_link_reassign["end_time"] = 2
df_fourgram_link_reassign["start_time"] = 1
df_fourgram_link_reassign["end_time"] = 2

In [124]:
df_twogram_link_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link_Temp.xlsx", index=False)
df_threegram_link_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link_Temp.xlsx", index=False)
df_fourgram_link_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link_Temp.xlsx", index=False)

In [125]:
# read file after youtube tool result
df_twogram_link_reassign_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link_Temp.xlsx")  # from youtube tool
df_twogram_link_reassign_video_list = list(set(df_twogram_link_reassign_youtube_tool["video_id"]))

In [126]:
df_threegram_link_reassign_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link_Temp.xlsx")  # from youtube tool
df_threegram_link_reassign_video_list = list(set(df_threegram_link_reassign_youtube_tool["video_id"]))

In [127]:
df_fourgram_link_reassign_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link_Temp.xlsx")  # from youtube tool
df_fourgram_link_reassign_video_list = list(set(df_fourgram_link_reassign_youtube_tool["video_id"]))

In [128]:
df_twogram_link_new = df_twogram_link[df_twogram_link["video_id"].isin(df_twogram_link_reassign_video_list)]
df_twogram_link_new

,search_string,start_time,end_time,sentence,video_id
3,fikir değil,1595,1597,aslında kötü de fikir değil,qWG21pLc2fw
7,fikir değil,21,23,fena fikir değil fena fikir değil,7oozYl45nzQ
8,fikir değil,555,557,fena fikir değil fena fikir değil,7oozYl45nzQ
9,fikir değil,20,22,aslında ne güzel fikir değil mi tabii eğer esn...,kqF6RxLJVtU
10,fikir değil,249,251,hmm fena fikir değil bunu bir düşünelim,MItD3hP_kqU
...,...,...,...,...,...
2279,yardım etmek,14395,14397,ihtiyaçların giderilmesine sessizlik ile yardı...,lHNtdUcq2_A
2280,yardım etmek,105,107,videolardan haberdar olmak ve daha fazla insan...,LM0aIRzrpVI
2283,yardım etmek,1400,1402,öğrenebilirsiniz her zaman her zaman size yard...,0Hyrz58zvcA
2287,yardım etmek,192,194,yardım etmek isteyen böyle aşırı merhametli in...,W5VdzH65XRU


In [129]:
df_threegram_link_new = df_threegram_link[df_threegram_link["video_id"].isin(df_threegram_link_reassign_video_list)]
df_threegram_link_new

,search_string,start_time,end_time,sentence,video_id
8,peki ya sen,136,139,aa peki ya sen,XK7yxtkkMhQ
13,peki ya sen,38,40,yarın ı düşlemekten ötürü peki ya sen nasılsın...,A33c4JiRDgM
17,sen neden bahsediyorsun,951,954,evetondan bahsediyorum sen neden bahsediyorsun...,FxRayyob69s
20,bir fikir değil,331,333,aslında bu çok da yeni bir fikir değil,WzNA02gjcPE
21,bir şey mi,685,707,gördüğünüz gibi bir şey mi,JzqvVxN6NJk
...,...,...,...,...,...
2029,böyle bir şey,197,199,ya böyle bir şey yok,5EsyiXMKOPs
2030,böyle bir şey,495,496,allaha şükür bugüne kadar böyle bir şey başımı...,6rqwPW97HiY
2031,böyle bir şey,895,897,eskiden gitmiyordum ben mesela böyle bir şey v...,h93wXNLXmYU
2032,böyle bir şey,66,68,alıyorum şu hayır öyle değilim çok amatörüm il...,duaQwKTmgvE


In [130]:
df_fourgram_link_new = df_fourgram_link[df_fourgram_link["video_id"].isin(df_fourgram_link_reassign_video_list)]
df_fourgram_link_new

,search_string,start_time,end_time,sentence,video_id
6,bana yardım eder misin,309,313,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
10,bana yardım eder misin,78,80,şu oyuncağı bir türlü kaldıramıyorum bana yard...,LMHUJ60Fsho
14,bir şeyin var mı,27,30,dostum bir şeyin var mı,FzHO9BqY_Vc
21,bir şeyin var mı,1661,1664,süper kaç yaşında emekli olmayı düşünüyorsun ö...,Ssg7vpw8Zb4
25,için her şeyi yaparım,2107,2110,yani arkadaşlarım için her şeyi yaparım olağan...,EXx3vjk8v0Y
...,...,...,...,...,...
770,başka bir şey var,649,651,söylemek istediğin başka bir şey var mı,miIgOGcO2tY
772,başka bir şey var,165,167,bu derece pervasız bir ruh halini motive eden ...,zRpBEE0OT1I
776,başka bir şey var,535,537,başka bir şey var mı hadi bey bunların dışında...,wsUGDArhZFg
777,başka bir şey var,813,816,söylemek istediğin başka bir şey var mı,vgHcue8Uu9I


In [131]:
df_twogram_link_new.to_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link.xlsx", index=False)
df_threegram_link_new.to_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link.xlsx", index=False)
df_fourgram_link_new.to_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link.xlsx", index=False)

#### Research New

In [20]:
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/\
Manuel Select/Revised Ngrams All in One.xlsx"
sheet = "Sheet2"

In [21]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,NaN,NaN
2,bana yardım eder misin,yardım eder misin,eder misin
3,bana yardım eder misin,yardım eder misin,yardım eder
4,başka bir şey daha,başka bir şey,bir şey
...,...,...,...
495,NaN,çok kötü bir,çok kötü
496,NaN,bir şey olmaz,NaN
497,NaN,şuna bir bak,bir bak
498,NaN,şuna bir bak,NaN


###### Twogram

In [22]:
df_twogram_link_new = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_Youtube_Link_Manuel.xlsx")
df_twogram_link_new

,video_id,start_time,end_time,text
0,NtoRI5nlwKw,1073,1075,duyuyor musun
1,CwuGepQDBrM,153,155,güzel bir
2,UQDUQnVUtq4,942,944,güzel bir
3,dRCz3FhBk08,97,99,güzel bir
4,7XVwNBlTIWs,13574,13576,güzel bir
...,...,...,...,...
525,XjIzyvw_UMA,157,159,daha önce
526,OAHJQ3_lKds,178,180,daha önce
527,Yr2ZLfWT0i8,4923,4925,daha önce
528,w312GmKmLFk,14,16,daha önce


In [23]:
#df_twogram_not_result = pd.DataFrame(set(df_file["twogram"].dropna()).difference(set(df_twogram_link_new["search_string"])), columns=["not_result"])
df_twogram_not_result = pd.DataFrame(set(df_file["twogram"].dropna()).difference(set(df_twogram_link_new["text"])), columns=["not_result"])
df_twogram_not_result

,not_result
0,fikir değil
1,kadar yeter
2,nerede olduğunu
3,yardım et
4,yaptığını sanıyorsun
5,biliyor musun
6,iyi misin
7,her şeyi
8,ne düşünüyorum
9,şey ister


In [24]:
#df_twogram_count = df_twogram_link_new.groupby(["search_string"])[["search_string"]].count()
df_twogram_count = df_twogram_link_new.groupby(["text"])[["text"]].count()
#df_twogram_count.rename(columns={"search_string":"count"}, inplace=True)
df_twogram_count.rename(columns={"text":"count"}, inplace=True)
df_twogram_count.sort_values(by="count", ascending=False, inplace=True)
df_twogram_count.reset_index(inplace=True)
df_twogram_count

,text,count
0,herhangi bir,8
1,ya da,8
2,daha önce,8
3,güzel bir,7
4,bir tane,7
...,...,...
154,neydi böyle,1
155,eder misin,1
156,seni seviyorum,1
157,yok değil,1


In [25]:
df_twogram_additional = df_twogram_count[df_twogram_count["count"] < 3]
#df_twogram_additional = df_twogram_additional.rename(columns={"search_string":"not_result"})
df_twogram_additional = df_twogram_additional.rename(columns={"text":"not_result"})
df_twogram_additional.drop("count", axis=1, inplace=True)
df_twogram_additional

,not_result
96,istiyor musun
97,tamam mı
98,ne güzel
99,istemiyor musun
100,hiç bir
...,...
154,neydi böyle
155,eder misin
156,seni seviyorum
157,yok değil


In [26]:
df_twogram_new_search = pd.concat([df_twogram_not_result, df_twogram_additional], axis=0)
df_twogram_new_search

,not_result
0,fikir değil
1,kadar yeter
2,nerede olduğunu
3,yardım et
4,yaptığını sanıyorsun
...,...
154,neydi böyle
155,eder misin
156,seni seviyorum
157,yok değil


In [27]:
df_twogram_new_search.to_excel(f"{out_path}/Twogram_Re-Search_Result.xlsx", index=False)

###### Threegram

In [28]:
df_threegram_link_new = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_Youtube_Link_Manuel.xlsx")
df_threegram_link_new

,video_id,start_time,end_time,text
0,7L64XN3or3I,30,32,hiçbir fikrim yok
1,yfbaI_fKgQc,230,232,hiçbir fikrim yok
2,OtTPQ4877Hk,462,464,hiçbir fikrim yok
3,xagFSGk8vT4,371,373,hiçbir fikrim yok
4,L3bf7dlR5hk,384,386,hiçbir fikrim yok
...,...,...,...,...
399,jAq0vdNeJ8c,383,385,için çok teşekkür
400,8iUkBU1vsDw,569,571,için çok teşekkür
401,fvuHfBDV7DU,435,437,için çok teşekkür
402,KSs8rK9tO04,340,342,iyi bir fikir


In [29]:
#df_threegram_not_result = pd.DataFrame(set(df_file["threegram"].dropna()).difference(set(df_threegram_link_new["search_string"])), columns=["not_result"])
df_threegram_not_result = pd.DataFrame(set(df_file["threegram"].dropna()).difference(set(df_threegram_link_new["text"])), columns=["not_result"])
df_threegram_not_result

,not_result
0,yapıyorsun değil mi
1,hepsi bu kadar
2,bana bir şey
3,var değil mi
4,bir şey olmayacak
5,şey söyledi mi
6,ne işin var
7,fark eder ki
8,hiç iyi değil
9,da ne böyle


In [30]:
#df_threegram_count = df_threegram_link_new.groupby(["search_string"])[["search_string"]].count()
df_threegram_count = df_threegram_link_new.groupby(["text"])[["text"]].count()
#df_threegram_count.rename(columns={"search_string":"count"}, inplace=True)
df_threegram_count.rename(columns={"text":"count"}, inplace=True)
df_threegram_count.sort_values(by="count", ascending=False, inplace=True)
df_threegram_count.reset_index(inplace=True)
df_threegram_count

,text,count
0,çok daha iyi,7
1,ama yine de,6
2,ne yazık ki,6
3,bu doğru mu,6
4,bir şey olmaz,6
...,...,...
138,biliyorsun değil mi,1
139,sana yardım etmek,1
140,sen de biliyorsun,1
141,sen de kimsin,1


In [31]:
df_threegram_additional = df_threegram_count[df_threegram_count["count"] < 3]
df_threegram_additional = df_threegram_additional.rename(columns={"text":"not_result"})
df_threegram_additional.drop("count", axis=1, inplace=True)
df_threegram_additional

,not_result
74,her şey yolunda
75,bunu daha önce
76,bu çok saçma
77,ne olduğunu biliyor
78,ne olursa olsun
...,...
138,biliyorsun değil mi
139,sana yardım etmek
140,sen de biliyorsun
141,sen de kimsin


In [32]:
df_threegram_new_search = pd.concat([df_threegram_not_result, df_threegram_additional], axis=0)
df_threegram_new_search

,not_result
0,yapıyorsun değil mi
1,hepsi bu kadar
2,bana bir şey
3,var değil mi
4,bir şey olmayacak
...,...
138,biliyorsun değil mi
139,sana yardım etmek
140,sen de biliyorsun
141,sen de kimsin


In [33]:
df_threegram_new_search.to_excel(f"{out_path}/Threegram_Re-Search_Result.xlsx", index=False)

###### Fourgram

In [34]:
df_fourgram_link_new = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Fourgram_With_{word_end}_Word_Youtube_Link_Manuel.xlsx")
df_fourgram_link_new

,video_id,start_time,end_time,text
0,3cAaBZ_Aml8,74,76,neden bu kadar çok
1,yOjfgHwn9y8,26,28,neden bu kadar çok
2,yK7PsjTRfwY,246,248,neden bu kadar çok
3,qaZnDHacYSk,515,517,neden bu kadar çok
4,WM2W7dCq3tA,981,983,neden bu kadar çok
...,...,...,...,...
170,VmFZ5g6Zeyw,17,19,bunu daha önce de
171,C6NzdthV39E,458,460,başka bir şey yok
172,66Oih2dmURc,1774,1774,başka bir şey yok
173,BfX0BJRrRhI,4106,4108,böyle bir şey olmayacak


In [35]:
#df_fourgram_not_result = pd.DataFrame(set(df_file["fourgram"].dropna()).difference(set(df_fourgram_link_new["search_string"])), columns=["not_result"])
df_fourgram_not_result = pd.DataFrame(set(df_file["fourgram"].dropna()).difference(set(df_fourgram_link_new["text"])), columns=["not_result"])
df_fourgram_not_result

,not_result
0,ne demek biliyor musun
1,bana yardım eder misin
2,seni görmek ne güzel
3,bunu yapmak zorunda değilsin
4,neden bu kadar uzun
5,bu hiç iyi değil
6,için her şeyi yaparım
7,bir şey ister misin
8,iyi bir fikrim var
9,bu da neydi böyle


In [36]:
#df_fourgram_count = df_fourgram_link_new.groupby(["search_string"])[["search_string"]].count()
df_fourgram_count = df_fourgram_link_new.groupby(["text"])[["text"]].count()
#df_fourgram_count.rename(columns={"search_string":"count"}, inplace=True)
df_fourgram_count.rename(columns={"text":"count"}, inplace=True)
df_fourgram_count.sort_values(by="count", ascending=False, inplace=True)
df_fourgram_count.reset_index(inplace=True)
df_fourgram_count

,text,count
0,onun gibi bir şey,6
1,bu da ne demek,6
2,bir şey söylemek istiyorum,6
3,bir şey değil mi,5
4,pek bir şey yok,5
...,...,...
62,bir şeyin var mı,1
63,bir şey söyledi mi,1
64,ben de seni seviyorum,1
65,başka bir şey var,1


In [37]:
df_fourgram_additional = df_fourgram_count[df_fourgram_count["count"] < 3]
#df_fourgram_additional = df_fourgram_additional.rename(columns={"search_string":"not_result"})
df_fourgram_additional = df_fourgram_additional.rename(columns={"text":"not_result"})
df_fourgram_additional.drop("count", axis=1, inplace=True)
df_fourgram_additional

,not_result
32,başka bir şey yok
33,ne var biliyor musun
34,ne olduğunu biliyor musun
35,bir şey daha var
36,kötü bir şey mi
37,bir şey söyleyeyim mi
38,sana bir şey söyleyeyim
39,diye bir şey yoktur
40,hiç böyle bir şey
41,hiç bir şey yok


In [38]:
df_fourgram_new_search = pd.concat([df_fourgram_not_result, df_fourgram_additional], axis=0)
df_fourgram_new_search

,not_result
0,ne demek biliyor musun
1,bana yardım eder misin
2,seni görmek ne güzel
3,bunu yapmak zorunda değilsin
4,neden bu kadar uzun
...,...
62,bir şeyin var mı
63,bir şey söyledi mi
64,ben de seni seviyorum
65,başka bir şey var


In [39]:
df_fourgram_new_search.to_excel(f"{out_path}/Fourgram_Re-Search_Result.xlsx", index=False)